In [ ]:
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.models import load_model
import requests
from bs4 import BeautifulSoup
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import imutils
import argparse
from scipy.spatial.distance import euclidean
from imutils.perspective import four_point_transform

In [ ]:
model = load_model('/content/recognition.h5')

In [ ]:

labels = {0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'cabbage', 4: 'capsicum', 5: 'carrot',
          6: 'cauliflower', 7: 'chilli pepper', 8: 'corn', 9: 'cucumber', 10: 'eggplant', 11: 'garlic', 12: 'ginger',
          13: 'grapes', 14: 'guava' , 15: 'jalepeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce',
          19: 'mango', 20: 'onion', 21: 'orange', 22: 'papaya', 23: 'paprika', 24: 'pear', 25: 'peas', 26: 'pineapple',
          27: 'pomegranate', 28: 'potato', 29: 'raddish', 30: 'soy beans', 31: 'spinach', 32: 'sweetcorn',
          33: 'sweetpotato', 34: 'tomato', 35: 'turnip', 36: 'watermelon'}

fruits = ['Apple', 'Banana' , 'Chilli Pepper', 'Grapes', 'Guava' , 'Jalepeno', 'Kiwi', 'Lemon', 'Mango', 'Orange', 'Papaya' ,
          'Paprika', 'Pear', 'Pineapple', 'Pomegranate', 'Watermelon']
vegetables = ['Beetroot', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Corn', 'Cucumber', 'Eggplant', 'Garlic' , 'Ginger',
              'Lettuce', 'Onion', 'Peas', 'Potato', 'Raddish', 'Soy Beans', 'Spinach', 'Sweetcorn', 'Sweetpotato',
              'Tomato', 'Turnip']


In [ ]:
def fetch_calories(prediction):
    try:
        url = 'https://www.google.com/search?&q=calories in ' + prediction
        req = requests.get(url).text
        scrap = BeautifulSoup(req, 'html.parser')
        calories = scrap.find("div", class_="BNeawe iBp4i AP7Wnd").text
        return calories
    except Exception as e:
        print(e)


In [ ]:
def prepare_image(img_path):
  img = load_img(img_path, target_size=(224, 224, 3))
  img = img_to_array(img)
  img = img / 255
  img = np.expand_dims(img, [0])
  answer = model.predict(img)
  y_class = answer.argmax(axis=-1)
  y = int(y_class[0])
  res = labels[y]
  return res.capitalize()

In [ ]:
def print_fresh(res):
    threshold_fresh = 1.00
    if res < threshold_fresh:
        print("The item is FRESH!")
    else:
        print("The item is ROTTEN!")


In [ ]:

def pre_proc_img(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (100, 100))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img


In [ ]:
def evaluate_rotten_vs_fresh(image_path):
    model1 = load_model('/content/rottenvsfresh.h5')

    prediction = model1.predict(pre_proc_img(image_path))

    return prediction[0][0]


In [ ]:
def size_of_object(img_path):
  image = cv2.imread(img_path)
  if image is None:
    print("Error: Unable to load image.")
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (9, 9), 0)
  edged = cv2.Canny(blur, 50, 100)
  edged = cv2.dilate(edged, None, iterations=1)
  edged = cv2.erode(edged, None, iterations=1)

  cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  (cnts, _) = contours.sort_contours(cnts)
  cnts = [x for x in cnts if cv2.contourArea(x) > 100]

  ref_object = cnts[0]
  box = cv2.minAreaRect(ref_object)
  box = cv2.boxPoints(box)
  box = np.array(box, dtype="int")
  box = perspective.order_points(box)
  (tl, tr, br, bl) = box
  dist_in_pixel = euclidean(tl, tr)
  dist_in_cm = 2
  pixel_per_cm = dist_in_pixel/dist_in_cm

  for cnt in cnts:
    box = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box
    cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
    mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2), tl[1] + int(abs(tr[1] - tl[1])/2))
    mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2), tr[1] + int(abs(tr[1] - br[1])/2))
    wid = euclidean(tl, tr)/pixel_per_cm
    ht = euclidean(tr, br)/pixel_per_cm
    cv2.putText(image, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(image, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
  cv2_imshow(image)

In [ ]:
def run():
    print("Fruits🍍-Vegetable🍅 Classification")
    img_path = input("Enter the path to the image file: ")
    img = Image.open(img_path).resize((250, 250))
    display(img)
    save_image_path = '/var/upload_images' + img_path.split('/')[-1]
    img.save(save_image_path)
    result = prepare_image(save_image_path)
    if result in vegetables:
        print('Category : Vegetables')
    else:
        print('Category : Fruit')
    print("Predicted:", result)
    cal = fetch_calories(result)
    if cal:
        print("Calories: " + cal)

    is_rotten = evaluate_rotten_vs_fresh(img_path)
    print(f'Prediction: {is_rotten}',print_fresh(is_rotten))
    print(f"")
    print(f"Size:")
    size_of_object(img_path)

In [ ]:
run()